# 4 - Iterators and Generators

## Manually Consuming an Iterator

In [1]:
# create a file to read
import os

text = '''
The cat
sat on
the mat
'''

file_name = "test_file.txt"

if os.path.isfile(file_name):
    os.remove(file_name)

with open(file_name, "w") as target_file:
    target_file.write(text)


In [2]:
with open(file_name) as f:
    try:
        while True:
            line = next(f)
            print(line, end='')
    except StopIteration:
        pass



The cat
sat on
the mat


In [3]:
items = [1, 2, 3]
it = iter(items)
it

In [5]:
next(it)

1

In [6]:
next(it)

2

In [7]:
next(it)

3

In [8]:
next(it)

StopIteration: 

## Delegating Iteration
Python’s iterator protocol requires __iter__() to return a special iterator object that implements a __next__() method to carry out the actual iteration. 

In [9]:
class Node:
    def __init__(self, value):
        self._value = value
        self._children = []

    def __repr__(self):
        return 'Node({!r})'.format(self._value)

    def add_child(self, node):
        self._children.append(node)

    def __iter__(self):
        return iter(self._children)


In [10]:
root = Node(0)
child1 = Node(1)
child2 = Node(2)
root.add_child(child1)
root.add_child(child2)
for ch in root:
    print(ch)

Node(1)
Node(2)


## Creating New Iteration Patterns with Generators
The mere presence of the yield statement in a function turns it into a generator. Unlike a normal function, a generator only runs in response to iteration. 

In [13]:
def frange(start, stop, increment):
    x = start
    while x < stop:
        yield x
        x += increment


In [25]:
frange(0, 8, 2)

<generator object frange at 0x0000028395B98FC0>

In [26]:
list(frange(0, 8, 2))

[0, 2, 4, 6]

In [27]:
gen = frange(0, 8, 2)
gen

<generator object frange at 0x0000028395B98CA8>

In [28]:
next(gen)

0

In [29]:
next(gen)

2

In [30]:
next(gen)

4

In [31]:
next(gen)

6

In [32]:
next(gen)

StopIteration: 

## Implementing the Iterator Protocol
By far, the easiest way to implement iteration on an object is to use a generator function.

In [34]:
class Node:
    def __init__(self, value):
        self._value = value
        self._children = []

    def __repr__(self):
        return 'Node({!r})'.format(self._value)

    def add_child(self, node):
        self._children.append(node)

    def __iter__(self):
        return iter(self._children)

    def depth_first(self):
        yield self
        for c in self:
            yield from c.depth_first()


In [35]:
root = Node(0)
child1 = Node(1)
child2 = Node(2)
root.add_child(child1)
root.add_child(child2)
child1.add_child(Node(3))
child1.add_child(Node(4))
child2.add_child(Node(5))
for ch in root.depth_first():
    print(ch)


Node(0)
Node(1)
Node(3)
Node(4)
Node(2)
Node(5)


In [36]:
root

Node(0)

In [39]:
for c in root:
    print(c)

Node(1)
Node(2)


In [42]:
print(root)
for i in root:
    print(i)
    for j in i:
        print(j)
    

Node(0)
Node(1)
Node(3)
Node(4)
Node(2)
Node(5)


##  Iterating in Reverse
Use the built-in reversed() function.

In [44]:
a = [1, 2, 3, 4]
list(reversed(a))

[4, 3, 2, 1]

In [45]:
file_name

'test_file.txt'

In [46]:
with open(file_name) as f:
    for line in reversed(list(f)):
        print(line, end='')


the mat
sat on
The cat



Reversed iteration can also be customized.

In [47]:
class Countdown:
    def __init__(self, start):
        self.start = start
 
    # Forward iterator
    def __iter__(self):
        n = self.start
        while n > 0:
            yield n
            n -= 1

    # Reverse iterator
    def __reversed__(self):
        n = 1
        while n <= self.start:
            yield n
            n += 1


In [49]:
c = Countdown(5)
c

In [50]:
list(c)

[5, 4, 3, 2, 1]

In [51]:
list(reversed(c))

[1, 2, 3, 4, 5]

## Defining Generator Functions with Extra State

In [52]:
from collections import deque

class linehistory:
    def __init__(self, lines, histlen=3):
        self.lines = lines
        self.history = deque(maxlen=histlen)
 
    def __iter__(self):
        for lineno, line in enumerate(self.lines,1):
            self.history.append((lineno, line))
            yield line

    def clear(self):
        self.history.clear()


In [53]:
file_name

'test_file.txt'

In [58]:
with open(file_name) as f:
    lines = linehistory(f)
    for line in lines:
        if 'cat' in line:
            for lineno, hline in lines.history:
                print('{}:{}'.format(lineno, hline), end='')


1:
2:The cat


## Taking a Slice of an Iterator

In [65]:
import itertools

def count(n):
    while True:
        yield n
        n += 1

c = count(0)
c

<generator object count at 0x0000028395BC4728>

In [66]:
[next(c) for _ in range(10)]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [67]:
next(c)

10

In [68]:
c = count(0)
x = itertools.islice(c, 10, 20)
x

In [69]:
list(x)

[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

## Skipping the First Part of an Iterable
Use the itertools.dropwhile() function...

In [72]:
mylist = list(range(1, 11))
mylist

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [76]:
import itertools

i = itertools.dropwhile(lambda x: x <= 5, mylist)
i

In [77]:
list(i)

[6, 7, 8, 9, 10]

##  Iterating Over All Possible Combinations or Permutations

In [79]:
import itertools

items = ['a', 'b', 'c']
perms = itertools.permutations(items)
perms

In [80]:
list(perms)

[('a', 'b', 'c'),
 ('a', 'c', 'b'),
 ('b', 'a', 'c'),
 ('b', 'c', 'a'),
 ('c', 'a', 'b'),
 ('c', 'b', 'a')]

In [82]:
list(itertools.combinations(items, 2))

[('a', 'b'), ('a', 'c'), ('b', 'c')]

In [83]:
list(itertools.combinations_with_replacement(items, 2))

[('a', 'a'), ('a', 'b'), ('a', 'c'), ('b', 'b'), ('b', 'c'), ('c', 'c')]

##  Iterating Over the Index-Value Pairs of a Sequence

In [84]:
items

['a', 'b', 'c']

In [87]:
for index, value in enumerate(items):
    print(index, "-> ",value)

0 ->  a
1 ->  b
2 ->  c


##  Iterating Over Multiple Sequences Simultaneously

In [90]:
a = ["a", "b", "c"]
b = ["x", "y", "z"]

zip(a, b)

In [91]:
list(zip(a, b))

[('a', 'x'), ('b', 'y'), ('c', 'z')]

In [89]:
for i, j in zip(a, b):
    print(i, j)

a x
b y
c z


We can use itertools.zip_longest() where one sequence is longer than the other.

In [92]:
import itertools

a = ["a", "b", "c", "d"]
b = ["x", "y", "z"]

itertools.zip_longest(a, b)

In [93]:
list(itertools.zip_longest(a, b))

[('a', 'x'), ('b', 'y'), ('c', 'z'), ('d', None)]

In [94]:
for i, j in itertools.zip_longest(a, b):
    print(i, j)

a x
b y
c z
d None


## Iterating on Items in Separate Containers

In [95]:
from itertools import chain

a, b

(['a', 'b', 'c', 'd'], ['x', 'y', 'z'])

In [97]:
list(itertools.chain(a, b))

['a', 'b', 'c', 'd', 'x', 'y', 'z']

##  Flattening a Nested Sequence

In [98]:
from collections import Iterable

def flatten(items, ignore_types=(str, bytes)):
    for x in items:
        if isinstance(x, Iterable) and not isinstance(x, ignore_types):
            yield from flatten(x)
        else:
            yield x


In [99]:
items = [1, 2, [3, 4, [5, 6], 7], 8]

In [101]:
list(flatten(items))

[1, 2, 3, 4, 5, 6, 7, 8]

## Iterating in Sorted Order Over Merged Sorted Iterables

In [103]:
import heapq

a = [1, 4, 7, 10]
b = [2, 5, 6, 11]
list(heapq.merge(a, b))

[1, 2, 4, 5, 6, 7, 10, 11]

In [105]:
import itertools

list(sorted(itertools.chain(a, b)))

[1, 2, 4, 5, 6, 7, 10, 11]

In [107]:
sorted(a + b)

[1, 2, 4, 5, 6, 7, 10, 11]

***